In [ ]:
# 1. 데이터 다운로드 및 압축 해제
!wget -O ivbag_seg.zip https://app.roboflow.com/ds/ZfowIIK4Qc?key=IS5NXyDmr9

import zipfile
with zipfile.ZipFile('/content/ivbag_seg.zip') as zf:
    zf.extractall('/content/ivbag_seg/')

# 2. YAML 파일 생성
!pip install -q PyYAML
import yaml

data = {
    'train': '/content/ivbag_seg/train/images',
    'val': '/content/ivbag_seg/valid/images',
    'test': '/content/ivbag_seg/test/images',
    'names': ['iv'],  # 클래스 이름 하나
    'nc': 1           # 클래스 수
}

with open('/content/ivbag_seg/iv.yaml', 'w') as f:
    yaml.dump(data, f)

# 3. YAML 확인
with open('/content/ivbag_seg/iv.yaml', 'r') as f:
    iv_yaml = yaml.safe_load(f)
    print(iv_yaml)

# 4. YOLO 설치 및 체크
!pip install -q ultralytics
import ultralytics
ultralytics.checks()

# 5. 세그멘테이션 모델 로드 및 학습
from ultralytics import YOLO

model = YOLO('yolov8s-seg.pt')  # 세그멘테이션용 YOLOv8s 모델 사용

model.train(
    data='/content/ivbag_seg/iv.yaml',
    epochs=50,
    patience=10,
    batch=8,
    imgsz=640,
    name='ivbag_seg_model'  # 💡 이 이름으로 runs 폴더에 저장됨
)

# 6. 테스트 이미지 목록 확인
from glob import glob

test_image_list = glob('/content/ivbag_seg/test/images/*')
test_image_list.sort()

for i, img_path in enumerate(test_image_list):
    print(f'i = {i}, {img_path}')

# 7. 테스트 이미지 예측 (결과 자동 저장됨)
results = model.predict(
    source='/content/ivbag_seg/test/images/',
    save=True
)

# 8. 모델 저장 (이름 변경해서 저장)
model.save('/content/ivbag_iv_seg.pt')  # 원하는 이름으로 저장
